## Auto mode

This is for testing propouses

In [1]:
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [2]:
gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

## No Simulator

In [5]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Visualization Parameters

In [6]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [7]:
simulator = Simulator(
	visualizers=[novis],
	gcs_sysids={'GCS': [1,2,3,4,5]},
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=[],
	supress_output=["veh","launcher"],#, , "logic", "proxy", "gcs"
	verbose=2,
)
orac = simulator.launch()

🚀 ArduPilot SITL vehicle 1 launched (PID 2187428)
🚀 ArduPilot SITL vehicle 5 launched (PID 2187440)
🚀 UAV logic for vehicle 1 launched (PID 2187429)
🚀 ArduPilot SITL vehicle 2 launched (PID 2187431)
🚀 UAV logic for vehicle 5 launched (PID 2187441)
🚀 ArduPilot SITL vehicle 4 launched (PID 2187437)
🚀 Proxy for vehicle 1 launched (PID 2187442)
🚀 ArduPilot SITL vehicle 3 launched (PID 2187452)
🚀 UAV logic for vehicle 3 launched (PID 2187454)
🚀 UAV logic for vehicle 2 launched (PID 2187443)
🚀 Proxy for vehicle 3 launched (PID 2187457)
🚀 UAV logic for vehicle 4 launched (PID 2187451)
🚀 Proxy for vehicle 5 launched (PID 2187450)
🚀 Proxy for vehicle 2 launched (PID 2187459)
🚀 Proxy for vehicle 4 launched (PID 2187461)
🚀 Starting Proxy 2
[Errno 111] Connection refused sleeping
🚀 Starting Proxy 5
🚀 Starting Proxy 3
🚀 Starting Proxy 1
🔗 UAV logic 2 is connected to Oracle ⚪
✅ Heartbeat received
Logic 🧠 2: 🚀 lauching
Vehicle 2: ▶️ Plan Started: 📋 auto
Vehicle 2: ▶️ Action Started: 📤 UPLOAD_MISSION


## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
Vehicle 2: 🧹 Cleared previous mission
Vehicle 2: ✅ Step Done: clear uav missions
Vehicle 2: ✅ 8 waypoints read.
Vehicle 2: 🧭 Mission[0] → cmd: WAYPOINT, x: -35.3631933, y: 149.1652241, z: 0.0, current: 0
Vehicle 2: 🧭 Mission[1] → cmd: DELAY, x: -35.3631933, y: 149.1652241, z: 0.0, current: 0
Vehicle 2: 🧭 Mission[2] → cmd: TAKEOFF, x: -35.3631933, y: 149.1652241, z: 5.0, current: 0
Vehicle 2: 🧭 Mission[3] → cmd: WAYPOINT, x: -35.3631933, y: 149.1651139, z: 5.0, current: 0
Vehicle 2: 🧭 Mission[4] → cmd: WAYPOINT, x: -35.3631034, y: 149.1651139, z: 5.0, current: 0
Vehicle 2: 🧭 Mission[5] → cmd: WAYPOINT, x: -35.3631034, y: 149.1652241, z: 5.0, current: 0
Vehicle 2: 🧭 Mission[6] → cmd: WAYPOINT, x: -35.3631933, y: 149.1652241, z: 5.0, current: 0
Vehicle 2: 🧭 Mission[7] → cmd: LAND, x: -35.3631933, y: 149.1652241, z: 0.0, current: 0
verb: 2
Vehicle 4: 🧹 Cleared previous mission
Vehicle 4: ✅ Step Do

📤 GCS ← UAV 5: Sending DONE (attempt 2)
📤 GCS ← UAV 5: Sending DONE (attempt 3)
📤 GCS ← UAV 5: Sending DONE (attempt 4)
GCS GCS: ✅ Vehicle 5 completed its mission
✅ All UAVs assigned to GCS GCS have completed their mission.
✅ ACK received. DONE message acknowledged.
❎ Vehicle 5 logic stopped.
💾 Trajectories saved to '/home/abeldg/uav-cyber-sim/data/trajectories_GCS.pkl'.
Connection reset or closed by peer on TCP socket
❎ Proxy 5 stopped.
